## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("./Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Port Hardy,CA,50.6996,-127.4199,38.75,93,90,5.75,light intensity shower rain
1,1,Mayo,US,38.8876,-76.5119,60.42,52,1,1.99,clear sky
2,2,Moose Factory,CA,51.2585,-80.6094,8.60,66,1,9.22,clear sky
3,3,Cherskiy,RU,68.7500,161.3000,-30.91,100,77,4.23,broken clouds
4,4,Chuy,UY,-33.6971,-53.4616,70.97,67,100,11.68,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
9,9,Nhulunbuy,AU,-12.2333,136.7667,81.21,83,40,0.00,scattered clouds
12,12,Kalabo,ZM,-14.9700,22.6814,79.66,46,89,2.08,overcast clouds
19,19,Matara,LK,5.9485,80.5353,76.86,90,93,5.03,overcast clouds
23,23,Saint-Philippe,RE,-21.3585,55.7679,78.39,82,14,10.49,few clouds
27,27,Atuona,PF,-9.8000,-139.0333,78.28,74,4,18.41,clear sky
28,28,Hithadhoo,MV,-0.6000,73.0833,81.72,78,78,12.64,broken clouds
35,35,Rikitea,PF,-23.1203,-134.9692,75.69,73,22,12.28,few clouds
42,42,Valparaiso,CL,-33.0393,-71.6273,80.83,48,0,8.05,clear sky
43,43,Vaini,TO,-21.2000,-175.2000,78.17,84,24,18.03,light rain
44,44,Kirakira,SB,-10.4544,161.9205,79.02,83,93,5.57,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                164
City                   164
Country                163
Lat                    164
Lng                    164
Max Temp               164
Humidity               164
Cloudiness             164
Wind Speed             164
Current Description    164
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
9,Nhulunbuy,AU,81.21,scattered clouds,-12.2333,136.7667,
12,Kalabo,ZM,79.66,overcast clouds,-14.9700,22.6814,
19,Matara,LK,76.86,overcast clouds,5.9485,80.5353,
23,Saint-Philippe,RE,78.39,few clouds,-21.3585,55.7679,
27,Atuona,PF,78.28,clear sky,-9.8000,-139.0333,
28,Hithadhoo,MV,81.72,broken clouds,-0.6000,73.0833,
35,Rikitea,PF,75.69,few clouds,-23.1203,-134.9692,
42,Valparaiso,CL,80.83,clear sky,-33.0393,-71.6273,
43,Vaini,TO,78.17,light rain,-21.2000,-175.2000,
44,Kirakira,SB,79.02,overcast clouds,-10.4544,161.9205,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
9,Nhulunbuy,AU,81.21,scattered clouds,-12.2333,136.7667,East Arnhem Real Estate
12,Kalabo,ZM,79.66,overcast clouds,-14.9700,22.6814,Sikakubete Guest House Kalabo Town Council
19,Matara,LK,76.86,overcast clouds,5.9485,80.5353,Amaloh Boutique Resort
23,Saint-Philippe,RE,78.39,few clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
27,Atuona,PF,78.28,clear sky,-9.8000,-139.0333,Villa Enata


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")


In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
max_temp = clean_hotel_df["Max Temp"]
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))